In [1]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.lookup_tools import extractor
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import crystalball
import matplotlib.pyplot as plt
from hist import Hist
from coffea.analysis_tools import PackedSelection
from scipy.integrate import quad

In [3]:
#TODO: fix path issue
fname = "NANOAOD-1500_900.root"
events = NanoEventsFactory.from_root(
    {fname: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DYJets"},
    delayed=False
).events()

/usr/local/lib/python3.10/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [4]:
def makeCutSet(x, s, *args):
    return [x[s > a] for a in args]

b_tag_wps = [0.0490, 0.2783, 0.7100]

def createObjects(events):
    good_jets = events.Jet[(events.Jet.pt > 30) & (abs(events.Jet.eta) < 2.4)]
    fat_jets = events.FatJet[(events.FatJet.pt > 30) & (abs(events.FatJet.eta) < 2.4)]
    loose_top, med_top, tight_top = makeCutSet(
        fat_jets, fat_jets.particleNet_TvsQCD, 0.58, 0.80, 0.97
    )
    #loose_W, med_W, tight_W = makeCutSet(
    #    fat_jets, fat_jets.particleNet_WvsQCD, 0.7, 0.94, 0.98
    #)

    #deep_top_wp1, deep_top_wp2, deep_top_wp3, deep_top_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_TvsQCD, 0.436, 0.802, 0.922, 0.989
    #)
    #deep_W_wp1, deep_W_wp2, deep_W_wp3, deep_W_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_WvsQCD, 0.458, 0.762, 0.918, 0.961
    #)
    loose_b, med_b, tight_b = makeCutSet(
        good_jets, good_jets.btagDeepFlavB, *(b_tag_wps[x] for x in range(3))
    )

    el = events.Electron
    good_electrons = el[
        (el.cutBased == 4)
        & (el.miniPFRelIso_all < 0.1)
        & (el.pt > 30)
        & (abs(el.eta) < 2.4)
    ]
    mu = events.Muon
    good_muons = mu[
        (mu.mediumId) & (mu.miniPFRelIso_all < 0.2) & (mu.pt > 30) & (abs(mu.eta) < 2.4)
    ]
    events["good_jets"] = good_jets
    events["good_electrons"] = good_electrons
    events["good_muons"] = good_muons

    events["loose_bs"] = loose_b
    events["med_bs"] = med_b
    events["tight_bs"] = tight_b

    events["tight_tops"] = tight_top
    # events["med_tops"] = med_top
    # events["loose_tops"] = loose_top

    # events["tight_Ws"] = tight_W
    # events["med_Ws"] = med_W
    # events["loose_Ws"] = loose_W

    # events["deep_top_wp1"] = deep_top_wp1
    # events["deep_top_wp2"] = deep_top_wp2
    # events["deep_top_wp3"] = deep_top_wp3
    # events["deep_top_wp4"] = deep_top_wp4

    # events["deep_W_wp1"] = deep_W_wp1
    # events["deep_W_wp2"] = deep_W_wp2
    # events["deep_W_wp3"] = deep_W_wp3
    # events["deep_W_wp4"] = deep_W_wp4

    return events

def createSelection(events):
    """Baseline selection for the analysis.
Applies the following selection:
- Jets[0].pt > 300
- 4 <= nJets <= 6
- 0 leptons
- 2 medium bjets, at least one of which is tight
- delta_R(med_bjets[0],med_bjets[1]) > 1
    """

    selection = PackedSelection()
    good_jets = events.good_jets
    fat_jets = events.FatJet
    good_muons = events.good_muons
    good_electrons = events.good_electrons
    loose_b = events.loose_bs
    med_b = events.med_bs
    tight_b = events.tight_bs
    tight_top = events.tight_tops
    # selection = PackedSelection()
    filled_jets = ak.pad_none(good_jets, 4, axis=1)
    top_two_dr = ak.fill_none(filled_jets[:, 0].delta_r(filled_jets[:, 1]), False)

    filled_med = ak.pad_none(med_b, 2, axis=1)
    med_dr = ak.fill_none(filled_med[:, 0].delta_r(filled_med[:, 1]), False)
    # selection.add("trigger", (ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6))
    if "HLT" in events.fields:
        selection.add("hlt", (events.HLT.PFHT1050 | events.HLT.AK8PFJet360_TrimMass30))
    selection.add("highptjet", (ak.fill_none(filled_jets[:, 0].pt > 300, False)))
    selection.add("jets", ((ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6)))
    selection.add("0Lep", ((ak.num(good_electrons) == 0) & (ak.num(good_muons) == 0)))
    selection.add("2bjet", (ak.num(med_b) >= 2))
    selection.add("1tightbjet", (ak.num(tight_b) >= 1))
    # selection.add("jet_dr", ((top_two_dr < 4) & (top_two_dr > 2)).to_numpy())
    selection.add("b_dr", (med_dr > 1))
    return selection

In [5]:
events = createObjects(events)
mask = createSelection(events).all("highptjet", "jets", "0Lep", "2bjet", "1tightbjet")
events = events[mask]

jetOrdinality,jetPT,jetEta,jetPhi,jetBScore,m3M,m3PT,m3Eta,m3Phi,m4M,m4PT,m4Eta,m4Phi,isStopMatched,isChiMatched,isOther

In [14]:
jets = events.Jet
gj = events.good_jets

In [25]:
ak.local_index(gj, axis=1)

<Array [[0, 1, 2, 3, 4, 5], [...], ..., [0, 1, 2, 3]] type='4469 * var * int64'>

In [31]:
def pad(arr):
            return ak.fill_none(
                ak.pad_none(arr, 100, axis=1, clip=True),
                0.0,
            )

In [58]:
imap = {
    "ordinality": ak.local_index(gj, axis=1),
    "pt": gj.pt,
    "eta": gj.eta,
    "phi": gj.phi,
    "bscore": gj.btagDeepFlavB,
    "m3_m": gj[:, :3].sum().mass,
    "m3_pt": gj[:, :3].sum().pt,
    "m3_eta": gj[:, :3].sum().eta,
    "m3_phi": gj[:, :3].sum().phi,
    "m4_m": gj[:, :4].sum().mass,
    "m4_pt": gj[:, :4].sum().pt,
    "m4_eta": gj[:, :4].sum().eta,
    "m4_phi": gj[:, :4].sum().phi
}

In [59]:
ordinality = ak.local_index(gj, axis=1)
pt = gj.pt
eta = gj.eta
phi = gj.phi
bscore = gj.btagDeepFlavB
m3_m = gj[:, :3].sum().mass
m3_pt = gj[:, :3].sum().pt
m3_eta = gj[:, :3].sum().eta
m3_phi = gj[:, :3].sum().phi
m4_m = gj[:, :4].sum().mass
m4_pt = gj[:, :4].sum().pt
m4_eta = gj[:, :4].sum().eta
m4_phi = gj[:, :4].sum().phi

In [61]:
ak.zip(imap)[0]

<Array [{ordinality: 0, pt: 448, ...}, ...] type='6 * {ordinality: int64, p...'>

In [56]:
(imap["m4_m"])[k]

1412.2499

In [27]:
imap

{'ordinality': <Array [[0, 1, 2, 3, 4, 5], [...], ..., [0, 1, 2, 3]] type='4469 * var * int64'>,
 'pt': <Array [[448, 359, 272, ..., 43.4, 39.8], ...] type='4469 * var * float32[p...'>,
 'eta': <Array [[0.138, -0.454, ..., -0.3, 2.05], ...] type='4469 * var * float32[p...'>,
 'phi': <Array [[-2.74, -0.685, ..., -0.858], ...] type='4469 * var * float32[param...'>,
 'bscore': <Array [[0.0232, 0.717, ..., 0.0298], ...] type='4469 * var * float32[param...'>,
 'm3_m': <Array [1.15e+03, 1.14e+03, ..., 1.06e+03, 2.6e+03] type='4469 * float32'>,
 'm3_pt': <Array [164, 228, 132, 112, 40, ..., 102, 58.2, 103, 265] type='4469 * float32'>,
 'm3_eta': <Array [0.25, 1.96, -0.841, 2.66, ..., 2.83, -1.03, 1.2] type='4469 * float32'>,
 'm3_phi': <Array [-2.18, 2.5, -1.3, ..., -1.56, 0.208, 0.0621] type='4469 * float32'>,
 'm4_m': <Array [1.39e+03, 1.41e+03, ..., 1.26e+03, 2.81e+03] type='4469 * float32'>,
 'm4_pt': <Array [110, 82.2, 99.2, 115, ..., 94.6, 48, 104, 111] type='4469 * float32'>,
 'm4_eta'